<a href="https://colab.research.google.com/github/ombuijabali/spatial_analysis/blob/main/spatial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install rasterstats

In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
import fiona
import os

In [ ]:
#Reading settelement boundary and adding a unique identifier

In [ ]:
# Path to the settlement polygons shapefile
settlements_path = '/content/drive/MyDrive/ML/spatial analysis/boundary/settelements.shp'
settlements_gdf = gpd.read_file(settlements_path)

# Add a new column 'settlement_id' with unique identifiers for each settlement
settlements_gdf['settlement_id'] = range(1, len(settlements_gdf) + 1)

# Save the GeoDataFrame back to the shapefile
settlements_gdf.to_file('/content/drive/MyDrive/ML/spatial analysis/boundary/settelements_with_id.shp')


In [ ]:
#Data Extraction for Education, Healthcare, Public Service and Business Centers

In [ ]:
#Read the unique classes in the data downloaded

In [ ]:
# Path to the pois polygon shapefile
input_shapefile_path = '/content/drive/MyDrive/ML/spatial analysis/landuse/gis_osm_pois_a_free_1.shp'

# Read the shapefile using geopandas
gdf = gpd.read_file(input_shapefile_path)

# Display the unique values in the "fclass" field
unique_classes = gdf['fclass'].unique()

# Print the unique classes
print("Unique Classes in the 'fclass' field:")
for fclass in unique_classes:
    print(fclass)

In [ ]:
#Extract the necessary classes to new shapefiles

In [ ]:
# Read the original shapefile
gdf = gpd.read_file(input_shapefile_path)

# Define the classes to extract
classes_to_extract = {
    'Education': ['school', 'college', 'university'],
    'Healthcare': ['hospital', 'pharmacy', 'clinic', 'doctors'],
    'Commercial': ['hotel', 'bank', 'supermarket', 'market_place'],
    'PublicServices': ['police', 'community_centre', 'library', 'fire_station']
}

# Create a new GeoDataFrame for each class and save to a new shapefile
for category, class_list in classes_to_extract.items():
    filtered_gdf = gdf[gdf['fclass'].isin(class_list)]

    if not filtered_gdf.empty:
        # Path for the new shapefile
        output_shapefile_path = f'/content/drive/MyDrive/ML/spatial analysis/results/{category}_shapefile.shp'

        # Save the filtered GeoDataFrame to a new shapefile
        filtered_gdf.to_file(output_shapefile_path)
        print(f"{category} Shapefile saved to {output_shapefile_path}")
    else:
        print(f"No features found for {category}")


In [ ]:
#Analyzing Population data for each settelement

In [ ]:
# Path to the population raster file
population_raster_path = '/content/drive/MyDrive/ML/spatial analysis/population/lbr_general_2020.tif'

# Path to the settlement polygons shapefile
settlements_path = '/content/drive/MyDrive/ML/spatial analysis/boundary/settelements_with_id.shp'

# Read the population raster using rasterio
with rasterio.open(population_raster_path) as population_raster:
    population_data = population_raster.read(1)
    transform = population_raster.transform

# Read the settlement polygons
settlements_gdf = gpd.read_file(settlements_path)

# Function to calculate zonal statistics (population sum)
def calculate_population_sum(polygon, population_data, transform):
    # Extract population sum using zonal_stats
    population_stats = zonal_stats(polygon.geometry, population_data, affine=transform, stats=['sum'])
    population_sum = population_stats[0]['sum']

    return pd.Series({'Population_Sum': population_sum})

# Apply the function to each settlement polygon
settlements_with_population_sum = settlements_gdf.apply(
    lambda row: calculate_population_sum(row, population_data, transform),
    axis=1
)

# Merge the results back to the original GeoDataFrame
settlements_with_population_sum = pd.concat([settlements_gdf, settlements_with_population_sum], axis=1)

# Save the GeoDataFrame with population sum information to a new shapefile
output_shapefile_path = '/content/drive/MyDrive/ML/spatial analysis/results/settlements_with_population_sum.shp'
settlements_with_population_sum.to_file(output_shapefile_path)

In [ ]:
#Find the number of buildings, schools, hosipital, markets for each settelement

In [ ]:
# Load school shapefile
schools = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/eductational/Education_shapefile.shp')

# Load hospital shapefile
hospitals = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/healthcare/Healthcare_shapefile.shp')

# Load public facilities shapefile
public_facilities = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/public_service/PublicServices_shapefile.shp')

# Load market shapefile
markets = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/commercial/Commercial_shapefile.shp')

# Load buildings shapefile
buildings = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/buildings/gis_osm_buildings_a_free_1.shp')

# Load boundary shapefile
boundaries = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/results/settlements_with_population_sum.shp')

# Perform spatial join for each type of facility
schools_within_boundaries = gpd.sjoin(schools, boundaries, how='inner', op='within')
hospitals_within_boundaries = gpd.sjoin(hospitals, boundaries, how='inner', op='within')
public_facilities_within_boundaries = gpd.sjoin(public_facilities, boundaries, how='inner', op='within')
markets_within_boundaries = gpd.sjoin(markets, boundaries, how='inner', op='within')
buildings_within_boundaries = gpd.sjoin(buildings, boundaries, how='inner', op='within')

# Group by boundary and count the number of facilities within each boundary
school_counts = schools_within_boundaries.groupby('index_right').size().reset_index(name='school_count')
hospital_counts = hospitals_within_boundaries.groupby('index_right').size().reset_index(name='hospital_count')
public_facilities_counts = public_facilities_within_boundaries.groupby('index_right').size().reset_index(name='public_count')
market_counts = markets_within_boundaries.groupby('index_right').size().reset_index(name='market_count')
buildings_counts = buildings_within_boundaries.groupby('index_right').size().reset_index(name='building_count')

# Merge the results back to the boundaries GeoDataFrame
boundaries_with_counts = boundaries.merge(school_counts, left_index=True, right_on='index_right', how='left')
boundaries_with_counts = boundaries_with_counts.merge(hospital_counts, left_on='index_right', right_on='index_right', how='left')
boundaries_with_counts = boundaries_with_counts.merge(public_facilities_counts, left_on='index_right', right_on='index_right', how='left')
boundaries_with_counts = boundaries_with_counts.merge(market_counts, left_on='index_right', right_on='index_right', how='left')
boundaries_with_counts = boundaries_with_counts.merge(buildings_counts, left_on='index_right', right_on='index_right', how='left')

# Fill NaN values (no facilities) with 0
boundaries_with_counts[['school_count', 'hospital_count', 'public_count', 'market_count', 'building_count']] = \
    boundaries_with_counts[['school_count', 'hospital_count', 'public_count', 'market_count', 'building_count']].fillna(0)

# Drop unnecessary columns
boundaries_with_counts.drop(['index_right'], axis=1, inplace=True)

# Save the result to a new shapefile
boundaries_with_counts.to_file('/content/drive/MyDrive/ML/spatial analysis/results/settlements_with_facility_count.shp', driver='ESRI Shapefile')

In [ ]:
#Find the distance between settelement and grid infrastructre

In [ ]:
#Install dependicies
pip install osmnx


In [ ]:
import geopandas as gpd
import osmnx as ox
import networkx as nx
from shapely.geometry import LineString

# Load the boundary shapefile
boundary = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/results/output11_shapefile.shp')

# Load the point shapefiles (power plants, power towers, substations)
power_plants = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/grid_infra/Liberia_PowerPlant.shp')
power_towers = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/grid_infra/Liberia_Powertower_withDEM.shp')
substations = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/grid_infra/Liberia_Substation.shp')

# Download road network graph for Liberia using osmnx
G = ox.graph_from_place('Liberia', network_type='drive')




In [ ]:
# Function to find the nearest node on the road network
def find_nearest_node(geometry, graph):
    if geometry.geom_type == 'Point':
        x, y = geometry.x, geometry.y
    elif geometry.geom_type == 'Polygon':
        x, y = geometry.centroid.x, geometry.centroid.y
    else:
        raise ValueError("Unsupported geometry type. Expected Point or Polygon.")

    node, distance = min(graph.nodes(data=True), key=lambda n: ((n[1]['x'] - x) ** 2 + (n[1]['y'] - y) ** 2) ** 0.5)
    return node, distance


# Function to calculate shortest distance for each point type
def calculate_shortest_distances(boundary_df, point_type, point_df):
    for b_idx, boundary_point in boundary_df.iterrows():
        # Find the nearest road network node for the boundary point
        nearest_node, boundary_distance = find_nearest_node(boundary_point.geometry, G)

        # Initialize variables to store the nearest point and its distance
        nearest_point, nearest_distance = None, float('inf')

        for p_idx, point in point_df.iterrows():
            # Find the nearest road network node for the target point
            target_node, distance = find_nearest_node(point.geometry, G)

            # Calculate shortest path length
            path_length = nx.shortest_path_length(G, source=nearest_node, target=target_node, weight='length')

            # Update nearest point if a shorter distance is found
            if path_length < nearest_distance:
                nearest_point, nearest_distance = point_type, path_length

        # Add the distance to the attribute table of the boundary shapefile
        boundary_df.at[b_idx, f'shortest_distance_to_{nearest_point}'] = nearest_distance

# Calculate shortest distances for each boundary point
calculate_shortest_paths(boundary, 'power_plants', power_plants)
calculate_shortest_paths(boundary, 'power_towers', power_towers)
calculate_shortest_paths(boundary, 'substations', substations)

# Save the updated boundary shapefile
boundary.to_file('/content/drive/MyDrive/ML/spatial analysis/results/settlement_shapefile_with_distances.shp')


In [ ]:
#Create a network of te extracted paths.

In [ ]:
# Initialize an empty GeoDataFrame to store the connecting lines
lines_gdf = gpd.GeoDataFrame(columns=['geometry', 'point_type', 'distance'])

# Function to calculate and store connecting lines
def calculate_connecting_lines(boundary_df, point_type, point_df):
    global lines_gdf  # Declare lines_gdf as global

    for b_idx, boundary_point in boundary_df.iterrows():
        # Find the nearest road network node for the boundary point
        nearest_node, boundary_distance = find_nearest_node(boundary_point.geometry, G)

        # Initialize variables to store the nearest point and its distance
        nearest_point, nearest_distance = None, float('inf')

        for p_idx, point in point_df.iterrows():
            # Find the nearest road network node for the target point
            target_node, distance = find_nearest_node(point.geometry, G)

            # Calculate shortest path
            path = nx.shortest_path(G, source=nearest_node, target=target_node, weight='length')

            # Convert the path to a LineString
            line = LineString([(G.nodes[node]['x'], G.nodes[node]['y']) for node in path])

            # Add the connecting line to the GeoDataFrame
            lines_gdf = lines_gdf.append({'geometry': line, 'point_type': point_type, 'distance': len(path)},
                                         ignore_index=True)

# Calculate connecting lines for each boundary point
calculate_connecting_lines(boundary, 'power_plants', power_plants)
calculate_connecting_lines(boundary, 'power_towers', power_towers)
calculate_connecting_lines(boundary, 'substations', substations)

# Save the GeoDataFrame with connecting lines to a new shapefile
lines_gdf.to_file('/content/drive/MyDrive/ML/spatial analysis/results/settlement_lines_shapefile.shp')


In [ ]:
#Additional data i.e Relative wealth index, soil/geology and landuse

In [ ]:
#Relative Wealth Index

In [ ]:
# Load rwi shapefile
rwi = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/rwi.shp')

# Load boundary shapefile
boundaries = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/distance_to_grid_infra.shp')

# Spatial join without common column names
rwi_within_boundaries = gpd.sjoin(rwi, boundaries, how='inner', op='within')

# Group by boundary and count the number of schools within each boundary
boundary_counts = rwi_within_boundaries.groupby('index_right').size().reset_index(name='rwi')

# Merge the result back to the boundaries GeoDataFrame
boundaries_with_rwi_counts = boundaries.merge(boundary_counts, left_index=True, right_on='index_right', how='left')

# Fill NaN values (no rwi) with 0
boundaries_with_rwi_counts['rwi'].fillna(0, inplace=True)

# Drop unnecessary columns
boundaries_with_rwi_counts.drop(['index_right'], axis=1, inplace=True)

# Display the result
print(boundaries_with_rwi_counts)

# Save the result to a new shapefile
boundaries_with_rwi_counts.to_file('/content/drive/MyDrive/ML/spatial analysis/results/output13_shapefile.shp', driver='ESRI Shapefile')


In [ ]:
#LandUse

In [ ]:
# Load the land use shapefile
land_use = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/landuse/gis_osm_landuse_a_free_1.shp')

# Load the boundary shapefile
boundary = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/results/output13_shapefile.shp')

# Perform spatial join
joined_data = gpd.sjoin(boundary, land_use, how="left", op="intersects")

# Reset the index of joined_data
joined_data = joined_data.reset_index(drop=True)

# Append the landuse column to the boundary shapefile
boundary['landuse'] = joined_data['fclass']

# Save the result to a new shapefile
boundary.to_file('/content/drive/MyDrive/ML/spatial analysis/results/boundary_land_use.shp')


In [ ]:
#Soil and Geology

In [ ]:
# Load the soil shapefile
soil = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/soil/soil.shp')

# Load the boundary shapefile
boundary = gpd.read_file('/content/drive/MyDrive/ML/spatial analysis/results/boundary_land_use.shp')

# Perform spatial join
joined_data = gpd.sjoin(boundary, soil, how="left", op="intersects")

# Reset the index of joined_data
joined_data = joined_data.reset_index(drop=True)

# Append the soil column to the boundary shapefile
boundary['soil_type'] = joined_data['DOMSOI']

# Save the result to a new shapefile
boundary.to_file('/content/drive/MyDrive/ML/spatial analysis/results/boundary_soil_geology.shp')


In [ ]:
#Visualization of results

In [ ]:
#Read the dataframe of final shapefile.

In [ ]:
import geopandas as gpd

# Path to the final shapefile
final_shapefile_path = '/content/drive/MyDrive/ML/spatial analysis/boundary_soil_geology.shp'

# Read the GeoDataFrame from the shapefile
final_gdf = gpd.read_file(final_shapefile_path)

# Display the GeoDataFrame
print(final_gdf.head())


In [ ]:
#Create Pie charts and a bar chart

In [ ]:
pip install plotly geopandas


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Load the shapefile
shapefile_path = '/content/drive/MyDrive/ML/spatial analysis/boundary_soil_geology.shp'
gdf = gpd.read_file(shapefile_path)

# Plot the bar chart
gdf.plot(kind='bar', x='settlement', y='school_cou', legend=False)
plt.xlabel('Settlements')
plt.ylabel('Number of Schools')
plt.title('Number of Schools in Each Settlement')
plt.show()


# Plot pie chart for 'landuse'
landuse_counts = gdf['landuse'].value_counts()
plt.figure(figsize=(7, 7))
plt.pie(landuse_counts, labels=landuse_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Land Use Distribution')
plt.show()

# Plot pie chart for 'soil_type'
soil_type_counts = gdf['soil_type'].value_counts()
plt.figure(figsize=(7, 7))
plt.pie(soil_type_counts, labels=soil_type_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Soil Type Distribution')
plt.show()

